In [50]:
import asyncio
import sys
import time
import logging
import inspect
import typing as ty

import aioitertools
from langchain.input import get_color_mapping
from langchain.agents import AgentExecutor, Tool
from langchain.agents.agent import AgentAction, AgentFinish
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun, Callbacks, CallbackManager, AsyncCallbackManager
)
from langchain.utilities.asyncio import asyncio_timeout
from langchain.load.dump import dumpd
sys.path.insert(0, "..")
from codeine.chatbot import build_chat_engine, service_context


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

chat_engine = build_chat_engine()

In [51]:
class AgentExecutorIterator:
    def __init__(
        self,
        agent_executor,
        inputs,
        callbacks=None,
        *,
        tags: list[str] | None = None,
        include_run_info: bool = False,
        async_: bool =False
    ):
        self.agent_executor = agent_executor
        self.inputs = inputs
        self.callbacks = callbacks
        self.iterations = 0
        self.time_elapsed = 0.0
        self.start_time = time.time()
        self.intermediate_steps = []
        self.async_ = async_
        self.include_run_info = include_run_info
        self.callback_manager = self.build_callback_manager(callbacks, tags=tags)
        
    def build_callback_manager(
        self,
        callbacks: Callbacks | None = None,
        *,
        tags: list[str] | None
    ) -> AsyncCallbackManager | CallbackManager:
        """
        Since we support sync and async, we build once we know what kind 
        of iterator we are using
        """
        CallbackMgr = AsyncCallbackManager if self.async_ else CallbackManager
        return CallbackMgr.configure(
            callbacks,
            self.agent_executor.callbacks,
            self.agent_executor.verbose,
            tags,
            self.agent_executor.tags
        )

    def __iter__(self):
        return self

    def __next__(self) -> dict[str, ty.Any]:
        """Iterator for hooking into agent steps"""
        self.run_manager = self.callback_manager.on_chain_start(
            dumpd(self.agent_executor),
            self.inputs,
        )
        # Construct a mapping of tool name to tool for easy lookup
        name_to_tool_map = {tool.name: tool for tool in self.agent_executor.tools}
        # We construct a mapping from each tool to a color, used for logging.
        color_mapping = get_color_mapping(
            [tool.name for tool in self.agent_executor.tools],
            excluded_colors=["green", "red"]
        )
        intermediate_steps: list[tuple[AgentAction, str]] = []
        # Let's start tracking the number of iterations and time elapsed
        iterations = 0
        time_elapsed = 0.0
        start_time = time.time()
        # Agent loop
        if not self.agent_executor._should_continue(self.iterations, self.time_elapsed):
            output = self.agent_executor.agent.return_stopped_response(
                self.agent_executor.early_stopping_method,
                self.intermediate_steps,
                **self.inputs
            )
            output = self.agent_executor._return(
                output, self.intermediate_steps, run_manager=self.run_manager
            )
            if self.run_manager:
                self.run_manager.on_chain_end(output)
            raise StopIteration(output)

        next_step_output = self.agent_executor._take_next_step(
            name_to_tool_map,
            color_mapping,
            self.inputs,
            self.intermediate_steps,
            run_manager=self.run_manager,
        )

        if isinstance(next_step_output, AgentFinish):
            output = self.agent_executor._return(
                next_step_output, self.intermediate_steps, run_manager=self.run_manager
            )
            raise StopIteration(output)

        intermediate_steps.extend(next_step_output)

        # Check for tool return
        if len(next_step_output) == 1:
            next_step_action = next_step_output[0]
            tool_return = self.agent_executor._get_tool_return(next_step_action)
            if tool_return is not None:
                output = self.agent_executor._return(
                    tool_return, self.intermediate_steps, run_manager=self.run_manager
                )
                raise StopIteration(output)

        output = {"intermediate_steps": intermediate_steps}

        iterations += 1
        time_elapsed = time.time() - start_time

        if self.run_manager:
            self.run_manager.on_chain_end(output)
        
        return output
        
    async def __aiter__(self):
        return self
        
    async def __anext__(self) -> dict[str, ty.Any]:
        """Async iterator for hooking into agent steps"""
        self.run_manager = await self.callback_manager.on_chain_start(
            dumpd(self.agent_executor),
            self.inputs,
        )
        # Construct a mapping of tool name to tool for easy lookup
        name_to_tool_map = {tool.name: tool for tool in self.agent_executor.tools}
        # We construct a mapping from each tool to a color, used for logging.
        color_mapping = get_color_mapping(
            [tool.name for tool in self.agent_executor.tools],
            excluded_colors=["green", "red"]
        )
        intermediate_steps: list[tuple[AgentAction, str]] = []
        # Let's start tracking the number of iterations and time elapsed
        iterations = 0
        time_elapsed = 0.0
        start_time = time.time()
        # Agent loop
        if not self.agent_executor._should_continue(self.iterations, self.time_elapsed):
            output = self.agent_executor.agent.return_stopped_response(
                self.agent_executor.early_stopping_method,
                self.intermediate_steps,
                **self.inputs
            )
            output = await self.agent_executor._areturn(
                output, self.intermediate_steps, run_manager=self.run_manager
            )
            if self.run_manager:
                await self.run_manager.on_chain_end(output)
            raise StopAsyncIteration(output)

        next_step_output = await self.agent_executor._atake_next_step(
            name_to_tool_map,
            color_mapping,
            inputs,
            intermediate_steps,
            run_manager=self.run_manager,
        )

        if isinstance(next_step_output, AgentFinish):
            output = await self.agent_executor._areturn(
                next_step_output, intermediate_steps, run_manager=self.run_manager
            )
            raise StopAsyncIteration(output)

        intermediate_steps.extend(next_step_output)

        # Check for tool return
        if len(next_step_output) == 1:
            next_step_action = next_step_output[0]
            tool_return = self.agent_executor._get_tool_return(next_step_action)
            if tool_return is not None:
                output = await self.agent_executor._areturn(
                    tool_return, intermediate_steps, run_manager=self.run_manager
                )
                raise StopAsyncIteration(output)

        output = {"intermediate_steps": intermediate_steps}

        iterations += 1
        time_elapsed = time.time() - start_time
        
        if self.run_manager:
            await self.run_manager.on_chain_end(output)
            
        return output

In [52]:
class MyAgentExecutor(AgentExecutor):
    def __call__(
        self,
        inputs: dict[str, ty.Any] | ty.Any,
        return_only_outputs: bool = False,
        callbacks: Callbacks = None,
        *,
        tags: list[str] | None = None,
        include_run_info: bool = False,
        iterator: bool = False,
        async_: bool = False,
    ) -> dict[str, ty.Any]:
        if iterator:
            return AgentExecutorIterator(
                self,
                inputs,
                callbacks,
                tags=tags,
                include_run_info=include_run_info,
                async_=async_
            )    
        else:
            return super().__call__(
                inputs,
                return_only_outputs,
                callbacks,
                tags=tags,
                include_run_info=include_run_info
            )

In [53]:
mae = MyAgentExecutor.from_agent_and_tools(
    agent=chat_engine._agent.agent,
    tools=chat_engine._agent.tools,
    callback_manager=chat_engine._agent.callback_manager
)
mae.memory = chat_engine._agent.memory
inspect.signature(mae.__call__)


<Signature (inputs: Union[dict[str, Any], Any], return_only_outputs: bool = False, callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None, *, tags: list[str] | None = None, include_run_info: bool = False, iterator: bool = False, async_: bool = False) -> dict[str, typing.Any]>

In [55]:
async for step in mae(inputs="What's the time?", iterator=True):
    print(step)

/tmp/ipykernel_23461/2231030265.py:1: RuntimeWarning: coroutine 'AgentExecutorIterator.__aiter__' was never awaited
  async for step in mae(inputs="What's the time?", iterator=True):


TypeError: 'async for' received an object from __aiter__ that does not implement __anext__: coroutine

In [ ]:
async def __aiter__(self):
    # Initialize instance variables for inputs, run_manager, iterations, time_elapsed, and start_time
    self.inputs = ...  # Pass the inputs when initializing the AgentExecutor
    self.run_manager = ...  # Pass the run_manager when initializing the AgentExecutor
    self.iterations = 0
    self.time_elapsed = 0.0
    self.start_time = time.time()
    self.intermediate_steps = []
    return self
AgentExecutor.__aiter__ = __aiter__


async def __anext__(self) -> Dict[str, Any]:
    """Async iterator for hooking into agent steps"""
    # Initialization should be done in __aiter__ or another method
    # and instance variables should be used for inputs, run_manager, iterations, and time_elapsed

    # Construct a mapping of tool name to tool for easy lookup
    name_to_tool_map = {tool.name: tool for tool in self.tools}
    # We construct a mapping from each tool to a color, used for logging.
    color_mapping = get_color_mapping(
        [tool.name for tool in self.tools], excluded_colors=["green", "red"]
    )
    intermediate_steps: List[Tuple[AgentAction, str]] = []
    # Let's start tracking the number of iterations and time elapsed
    iterations = 0
    time_elapsed = 0.0
    start_time = time.time()
    # Agent loop
    if not self._should_continue(iterations, time_elapsed):
        output = self.agent.return_stopped_response(
            self.early_stopping_method, self.intermediate_steps, **self.inputs
        )
        output = await self._areturn(
            output, self.intermediate_steps, run_manager=self.run_manager
        )
        raise StopAsyncIteration(output)

    next_step_output = await self._atake_next_step(
        name_to_tool_map,
        color_mapping,
        inputs,
        intermediate_steps,
        run_manager=run_manager,
    )

    if isinstance(next_step_output, AgentFinish):
        output = await self._areturn(
            next_step_output, intermediate_steps, run_manager=run_manager
        )
        raise StopAsyncIteration(output)

    intermediate_steps.extend(next_step_output)
    
    # Check for tool return
    if len(next_step_output) == 1:
        next_step_action = next_step_output[0]
        tool_return = self._get_tool_return(next_step_action)
        if tool_return is not None:
            output = await self._areturn(
                tool_return, intermediate_steps, run_manager=run_manager
            )
            raise StopAsyncIteration(output)
            
    output = {"intermediate_steps": intermediate_steps}

    iterations += 1
    time_elapsed = time.time() - start_time

    return output

AgentExecutor.__anext__ = __anext__

In [5]:
async def intermediate_steps_generator(
    self,
    inputs: dict[str, str],
    run_manager: ty.Optional[AsyncCallbackManagerForChainRun] = None,
) -> ty.AsyncIterator[tuple[AgentAction, str]]:
    """Generator function that yields intermediate steps (thoughts, actions, and observations)."""

    logger.debug("Starting generator")

    name_to_tool_map = {tool.name: tool for tool in self.tools}
    color_mapping = get_color_mapping(
        [tool.name for tool in self.tools], excluded_colors=["green"]
    )
    intermediate_steps: List[Tuple[AgentAction, str]] = []
    iterations = 0
    time_elapsed = 0.0
    start_time = time.time()

    async with asyncio_timeout(self.max_execution_time):
        try:
            while self._should_continue(iterations, time_elapsed):
                logger.debug(f"Iteration {iterations}")

                next_step_output = await self._atake_next_step(
                    name_to_tool_map,
                    color_mapping,
                    inputs,
                    intermediate_steps,
                    run_manager=run_manager,
                )
                if isinstance(next_step_output, AgentFinish):
                    # Yield the final answer - Do I want to do this here?
                    final_answer = next_step_output.return_values["output"]
                    yield next_step_output, final_answer
                    logger.debug("Agent finished")
                    break

                intermediate_steps.extend(next_step_output)
                if len(next_step_output) == 1:
                    next_step_action = next_step_output[0]
                    tool_return = self._get_tool_return(next_step_action)
                    if tool_return is not None:
                        logger.debug("Tool returned")
                        break

                # Yield the latest intermediate step(s)
                for step_output in next_step_output:
                    logger.debug(f"Yielding step: {step_output}")
                    yield step_output

                iterations += 1
                time_elapsed = time.time() - start_time
        except TimeoutError:
            logger.debug("TimeoutError")
            pass
    logger.debug("Generator finished")

AgentExecutor.intermediate_steps_generator = intermediate_steps_generator

In [6]:
async def _acall(
    self,
    inputs: dict[str, str],
    run_manager: ty.Optional[AsyncCallbackManagerForChainRun] = None,
) -> dict[str, str]:
    """Run text through and get agent response."""
    intermediate_steps: list[tuple[AgentAction, str]] = []

    async for step in self.intermediate_steps_generator(inputs, run_manager):
        intermediate_steps.append(step)

    output = self.agent.return_stopped(intermediate_steps)
    return output

AgentExecutor._acall = _acall

In [7]:
async def _atake_next_step(
    self,
    name_to_tool_map: dict[str, Tool],
    color_mapping: dict[str, str],
    inputs: dict[str, str],
    intermediate_steps: list[tuple[AgentAction, str]],
    run_manager: ty.Optional[AsyncCallbackManagerForChainRun] = None,
) -> list[tuple[AgentAction, str]] | AgentFinish:
    """Take a single step in the thought-action-observation loop."""

    logger.debug("Taking next step")

    try:
        # Call the LLM to see what to do.
        output = await self.agent.aplan(
            intermediate_steps,
            callbacks=run_manager.get_child() if run_manager else None,
            **inputs,
        )
    except OutputParserException as e:
        if isinstance(self.handle_parsing_errors, bool):
            raise_error = not self.handle_parsing_errors
        else:
            raise_error = False
        if raise_error:
            raise e
        text = str(e)
        if isinstance(self.handle_parsing_errors, bool):
            if e.send_to_llm:
                observation = str(e.observation)
                text = str(e.llm_output)
            else:
                observation = "Invalid or incomplete response"
        elif isinstance(self.handle_parsing_errors, str):
            observation = self.handle_parsing_errors
        elif callable(self.handle_parsing_errors):
            observation = self.handle_parsing_errors(e)
        else:
            raise ValueError("Got unexpected type of `handle_parsing_errors`")
        output = AgentAction("_Exception", observation, text)
        if run_manager:
            run_manager.on_agent_action(output, color="green")
        tool_run_kwargs = self.agent.tool_run_logging_kwargs()
        observation = ExceptionTool().run(
            output.tool_input,
            verbose=self.verbose,
            color=None,
            callbacks=run_manager.get_child() if run_manager else None,
            **tool_run_kwargs,
        )
        return [(output, observation)]

    logger.debug(f"Agent output: {output}")

    # If the tool chosen is the finishing tool, then we end and return.
    if isinstance(output, AgentFinish):
        return output
    actions: List[AgentAction]
    if isinstance(output, AgentAction):
        actions = [output]
    else:
        actions = output

    async def _aperform_agent_action(
        agent_action: AgentAction,
    ) -> tuple[AgentAction, str]:
        if run_manager:
            await run_manager.on_agent_action(
                agent_action, verbose=self.verbose, color="green"
            )
        # Otherwise we lookup the tool
        if agent_action.tool in name_to_tool_map:
            tool = name_to_tool_map[agent_action.tool]
            return_direct = tool.return_direct
            color = color_mapping[agent_action.tool]
            tool_run_kwargs = self.agent.tool_run_logging_kwargs()
            if return_direct:
                tool_run_kwargs["llm_prefix"] = ""
            # We then call the tool on the tool input to get an observation
            observation = await tool.arun(
                agent_action.tool_input,
                verbose=self.verbose,
                color=color,
                callbacks=run_manager.get_child() if run_manager else None,
                **tool_run_kwargs,
            )
        else:
            tool_run_kwargs = self.agent.tool_run_logging_kwargs()
            observation = await InvalidTool().arun(
                agent_action.tool,
                verbose=self.verbose,
                color=None,
                callbacks=run_manager.get_child() if run_manager else None,
                **tool_run_kwargs,
            )
        return agent_action, observation

    # Use asyncio.gather to run multiple tool.arun() calls concurrently
    result = await asyncio.gather(
        *[_aperform_agent_action(agent_action) for agent_action in actions]
    )

    return list(result)


AgentExecutor._atake_next_step = _atake_next_step

<Signature (inputs: Union[dict[str, Any], Any], return_only_outputs: bool = False, callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None, *, tags: list[str] | None = None, include_run_info: bool = False, iterator: bool = False, async_: bool = False) -> dict[str, typing.Any]>

NameError: name 'CallbackManager' is not defined

In [23]:
chat_engine._agent.callback_manager

In [21]:
dir(chat_engine._agent)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_acall',
 '_areturn',
 '_atake_next_step',
 '_calculate_ke

In [8]:
chat_engine._agent.return_intermediate_steps = True
chat_engine._agent.verbose = False

In [19]:
chat_engine._agent()

TypeError: Chain.__call__() missing 1 required positional argument: 'inputs'

In [10]:
inputs = {
    "input": "How does TinyViT work?",
    "chat_history" : []
}

async for ix, step in aioitertools.enumerate(chat_engine._agent.intermediate_steps_generator(
    inputs,
)):
    logging.info("==========================================")
    logging.info(f"Step: {ix}")
    if isinstance(step, str): # agent is finished
        text = step
    else: # agent has a tool to use
        action, text = step
        logging.info(f"Action: {action}")
    logging.info(f"Text: {text}")
    logging.info("==========================================")



INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1362 request_id=08c2551c8e6d35c00324fb017c8f328d response_code=200
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 4 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 4427 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:root:==========================================
INFO:root:Step: 0
INFO:root:Action: AgentAction(tool='Codeine Source Code Search', tool_input='TinyViT', log='{\n    "action": "Codeine Source Code Search",\n    "action_input": "TinyViT"\n}')
INFO:root:Text: TinyViT is a neural network architecture designed for vision tasks, with a focus on being small and efficient. It consists of multiple transformer blocks with different depths, number 

In [64]:
step

"Training TinyViT, a smaller version of the Vision Transformer (ViT) model, involves a similar process to training the original ViT. The main steps include: 1. Preparing a dataset of images and their corresponding labels. 2. Initializing the TinyViT model with a smaller architecture compared to the original ViT. 3. Feeding the images through the model and computing the loss based on the model's predictions and the true labels. 4. Updating the model's weights using an optimization algorithm, such as Adam or SGD, to minimize the loss. 5. Repeating steps 3 and 4 for multiple epochs until the model converges or reaches a satisfactory performance level. The primary difference between TinyViT and the original ViT is the model's size, which makes TinyViT more efficient and faster to train, while still maintaining competitive performance on various computer vision tasks."

In [17]:
print(inspect.getsource(AgentExecutor))

class AgentExecutor(Chain):
    """Consists of an agent using tools."""

    agent: Union[BaseSingleActionAgent, BaseMultiActionAgent]
    tools: Sequence[BaseTool]
    return_intermediate_steps: bool = False
    max_iterations: Optional[int] = 15
    max_execution_time: Optional[float] = None
    early_stopping_method: str = "force"
    handle_parsing_errors: Union[
        bool, str, Callable[[OutputParserException], str]
    ] = False

    @classmethod
    def from_agent_and_tools(
        cls,
        agent: Union[BaseSingleActionAgent, BaseMultiActionAgent],
        tools: Sequence[BaseTool],
        callback_manager: Optional[BaseCallbackManager] = None,
        **kwargs: Any,
    ) -> AgentExecutor:
        """Create from agent and tools."""
        return cls(
            agent=agent, tools=tools, callback_manager=callback_manager, **kwargs
        )

    @root_validator()
    def validate_tools(cls, values: Dict) -> Dict:
        """Validate that tools are compatible with agen